# Und das Wort ist Fleisch geworden
* https://textmining.wp.hs-hannover.de/Preprocessing.html
* Tagset: https://homepage.ruhr-uni-bochum.de/stephen.berman/Korpuslinguistik/Tagsets-STTS.html

In [1]:
import pandas as pd
import nltk
import json
from pathlib import Path
from HanTa import HanoverTagger as ht
import glob
import math

# Do once
#nltk.download('punkt')

## Lemmatize / Tag and store as CSV (only once, then goto: next cell)
Store lemmatized and tagged data as json because of performance issues. Loading a csv with (many!) dicts is very slow and like flying from BER...  
Split it up to store it on github (max filesize = 100mb)

In [2]:
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

In [ ]:
df_votum_raw = pd.read_csv(Path('../export/votum.csv'))

# Remove empty texts
df_votum_raw = df_votum_raw[df_votum_raw.text.notna()]

In [24]:
# Start tagging
records = []
for i, row in df_votum_raw.iterrows():
    lemm = nltk.tokenize.word_tokenize(row['text'], language='german')

    records.append({
        'name': row['name'],
        'vorname': row['vorname'],
        'partei': row['partei'],
        'jahrgang': row['jahrgang'],
        'geschlecht': row['geschlecht'],
        'funktion': row['funktion'],
        'ismember': row['ismember'],
        'tags': tagger.tag_sent(lemm, taglevel=1),
    })

In [58]:

# Split
chunks = 5
bucketsize = math.ceil(len(records) / chunks)
for i in range(0, chunks):
    subrecords = []
    for j in range(bucketsize * i, bucketsize * ( i + 1 )):
        if j < len(records):
            subrecords.append(records[j])

    # Store
    with open(Path('../export/tags/tag_%s.json' % i), 'w', encoding='utf-8') as f:
        f.write(json.dumps(subrecords, ensure_ascii=False))

print("finito")

finito


## Load stemmatized Data

In [2]:
records = []
for f in glob.glob(str(Path('../export/tags/*.json'))):
    records = records + json.load(open(f, 'r', encoding='utf-8'))

print(len(records))

46521


In [3]:
# Members only, no Presidents
r_members = list(filter(lambda x: x['ismember'] == True, records))
r_members = list(filter(lambda x: x['funktion'] not in ['Präsidium', '2. Vizepräsidium', '1. Vizepräsidium'], r_members))
print(len(r_members))

34497


## Analyse

In [4]:
# Select only Nouns
list_m = []
list_w = []
for r in r_members:
    tags = [lemma for (word,lemma,pos) in r['tags'] if pos == "NN" or pos == "NE"]
    if r['geschlecht'] == 'm': list_m = list_m + tags
    elif r['geschlecht'] == 'w': list_w = list_w + tags

In [5]:
def get_freqdist_df(l):
    fdist = nltk.FreqDist(l)
    fdist = fdist.most_common(100)
    df_dist = pd.DataFrame(fdist, columns=['w', 'count'])
    df_dist.reset_index(drop=False, inplace=True)
    return df_dist

df_m = get_freqdist_df(list_m)
df_m['geschlecht'] = 'm'

df_w = get_freqdist_df(list_w)
df_w['geschlecht'] = 'w'

df_concat = pd.concat([df_w, df_m])

# Remove duplicates
df_concat = df_concat.drop_duplicates(subset=['w'], keep = False)

df_concat

,index,w,count,geschlecht
62,62,Angebot,1092,w
69,69,Umsetzung,1010,w
76,76,Jugendliche,968,w
79,79,Liste,942,w
81,81,Lehrpersonen,918,w
85,85,Eltern,898,w
86,86,Kef-erklärung,895,w
88,88,Forderung,890,w
91,91,Familie,849,w
94,94,Schüler,819,w


In [6]:
list(df_concat[df_concat.geschlecht == 'w']['w'])

['Angebot',
 'Umsetzung',
 'Jugendliche',
 'Liste',
 'Lehrpersonen',
 'Eltern',
 'Kef-erklärung',
 'Forderung',
 'Familie',
 'Schüler',
 'Absatz',
 'Usz',
 'Bildung',
 'Patient',
 'Grundlage']

In [7]:
list(df_concat[df_concat.geschlecht == 'm']['w'])

['Staat',
 'Sache',
 'Verkehr',
 'Weiss',
 'Wort',
 'Winterthur',
 'Regierungsrat',
 'Schluß',
 'Evp',
 'Geschäft',
 'Parlament',
 'Kollege',
 'Name',
 'Votum',
 'Debatte']